In [1]:
from itertools import product
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.utils.data as utils
from matplotlib import image as img
from matplotlib import pyplot as plt
import os
import pandas as pd

In [18]:
def train(net, X, y, optimizer, criterion, batch_size, lr, p=30):
    X = torch.Tensor(X).float()
    N = len(X)
    y = torch.Tensor(y).long()
    train_size = int(0.75 * N)
    valid_size = N - train_size
    train_index, valid_index = torch.utils.data.random_split(
        np.arange(N), [train_size, valid_size]
    )
    train_dataset = utils.TensorDataset(X[train_index], y[train_index])
    valid_dataset = utils.TensorDataset(X[valid_index], y[valid_index])
    dataloader = utils.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=2
    )
    validloader = utils.DataLoader(
        valid_dataset, batch_size=batch_size, shuffle=True, num_workers=2
    )
    j = 0
    epoch = 0
    train_accs = []
    valid_accs = []
    train_losses = []
    valid_losses = []
    best_vloss = float("inf")
    while j < p:
        epoch += 1
        net.train_mode()
        for batch in dataloader:
            optimizer.zero_grad()
            X, y = batch
            X = X.view(-1, 3, 64, 64)
            y = y.view(-1)
            X = X.cuda()
            y = y.cuda()

            loss = criterion(net.forward(X), y)
            loss.backward()
            optimizer.step()

        net.eval_mode()
        train_loss, train_acc = net.evaluate(dataloader, criterion)
        valid_loss, valid_acc = net.evaluate(validloader, criterion)

        train_accs.append(train_acc)
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        valid_accs.append(valid_acc)
        if valid_loss < best_vloss:
            best_vacc = valid_acc
            best_vloss = valid_loss
            best_net = net
            best_epoch = epoch
            j = 0
        else:
            j += 1

        print("epoch: {}".format(epoch))
        print(" [LOSS] TRAIN {} / VALID {}".format(train_loss, valid_loss))
        print(" [ACC] TRAIN {} / VALID {}".format(train_acc, valid_acc))
    best_model = {
        "net": best_net.state_dict(),
        "best_epoch": best_epoch,
        "final_epoch": epoch,
        "batch_size": batch_size,
        "lr": lr,
        "vacc": best_vacc,
        "vloss": valid_losses,
        "taccs": train_accs,
        "tloss": train_losses,
        "vaccs": valid_accs,
        "convs": net.conv_size,
        "lins": net.lin_size,
    }
    return best_model


def accuracy(y_pred, target):
    correct = torch.eq(y_pred.max(1)[1], target).sum().type(torch.FloatTensor)
    return correct / len(target)


class Flatten(nn.Module):
    def forward(self, x):
        x = x.view(x.size(0), -1)
        return x


class Dropout(nn.Module):
    def __init__(self, p=0.3):
        super(Dropout, self).__init__()
        self.p = p

    def forward(self, x):
        data = x.data
        shape = data.shape
        size = int(shape[0] * shape[1])
        drop_idx = np.random.choice(
            np.arange(size), replace=False, size=int(size * self.p)
        )
        data = data.flatten()
        data[drop_idx] = 0
        data = data.reshape(shape)
        x.data = data
        return x


class Net(nn.Module):
    def __init__(self, input_size, conv_size, lin_size):
        super(Net, self).__init__()

        self.drop = True
        self.lin_size = lin_size
        self.conv_size = conv_size
        self.model = nn.Sequential(
            nn.Conv2d(input_size, conv_size, 3, 1),
            nn.MaxPool2d(2, 2),
            nn.ReLU(True),
            nn.Conv2d(conv_size, conv_size * 2, 3, 1),
            nn.ReLU(True),
            nn.Conv2d(conv_size * 2, conv_size * 2, 3, 1),
            nn.ReLU(True),
            nn.MaxPool2d(3, 3),
            nn.ReLU(True),
            Flatten(),
            nn.Linear(conv_size * 162, lin_size)).cuda()
        self.dropout = nn.Sequential(Dropout()).cuda()
        self.lin = nn.Sequential(
            nn.ReLU(True),
            nn.Linear(lin_size, 2),
        ).cuda()

    def eval_mode(self):
        self.drop = False
        
    def train_mode(self):
        self.drop = True
        
    def forward(self, x):
        dat = self.model(x)
        if self.drop:
            dat = self.dropout(dat)
        return self.lin(dat)

    def evaluate(self, dataloader, criterion):
        self.eval_mode()
        LOSSES = 0
        ACCURACY = 0
        COUNTER = 0
        for batch in dataloader:
            X, y = batch
            X = X.view(-1, 3, 64, 64)
            y = y.view(-1)
            X = X.cuda()
            y = y.cuda()

            loss = criterion(self.forward(X), y)
            acc = accuracy(self.forward(X), y)
            n = y.size(0)
            LOSSES += loss.sum().data.cpu().numpy() * n
            ACCURACY += acc.sum().data.cpu().numpy() * n
            COUNTER += n

        floss = LOSSES / float(COUNTER)
        faccuracy = ACCURACY / float(COUNTER)
        return floss, faccuracy
    
    def train(self, X, y, epochs, optimizer, criterion, batch_size, lr):
        train_dataset = utils.TensorDataset(X, y)
        dataloader = utils.DataLoader(
            train_dataset, batch_size=batch_size, shuffle=True, num_workers=2
        )
        train_accs = []
        train_losses = []
        for e in epochs:
            net.train_mode()
            for batch in dataloader:
                optimizer.zero_grad()
                X, y = batch
                X = X.view(-1, 3, 64, 64)
                y = y.view(-1)
                X = X.cuda()
                y = y.cuda()

                loss = criterion(net.forward(X), y)
                loss.backward()
                optimizer.step()

            net.eval_mode()
            train_loss, train_acc = net.evaluate(dataloader, criterion)

            train_accs.append(train_acc)
            train_losses.append(train_loss)

            print("epoch: {}".format(epoch))
            print("TRAIN : [LOSS] {} / [ACC] {}".format(train_loss, train_acc))
        print('Training Done')


def create_dataset(cats_path, dogs_path, test_path):
    train_set = []
    target = []
    n_cats = len(os.listdir(cats_path))
    for i in range(1, n_cats):
        f = '{}.Cat.jpg'.format(i)
        mat = img.imread(cats_path + f)
        if not len(mat.shape) == 3:
            mat = np.array((mat, mat, mat)).T
        train_set.append(mat)
        target.append(0)

    n_dogs = len(os.listdir(dogs_path))
    for i in range(1, n_dogs):
        f = '{}.Dog.jpg'.format(i)
        mat = img.imread(dogs_path + f)
        if not len(mat.shape) == 3:
            mat = np.array((mat, mat, mat)).T
        train_set.append(mat)
        target.append(1)

    train_set = np.asarray(train_set)
    target = np.asarray(target)
    
    test_set = []
    n_test = len(os.listdir(test_path))
    for i in range(1, n_test):
        f = '{}.jpg'.format(i)
        mat = img.imread(test_path + f)
        if not len(mat.shape) == 3:
            mat = np.array((mat, mat, mat)).T
        test_set.append(mat)

    return train_set, target, np.asarray(test_set)

In [9]:
test_path = "/home/arthur/git/IFT6135_assignment_1/testset/test/"
dogs_path = "/home/arthur/git/IFT6135_assignment_1/trainset/Dog/"
cats_path = "/home/arthur/git/IFT6135_assignment_1/trainset/Cat/"
X, y, submission = create_dataset(cats_path, dogs_path, test_path)
X = X / 255.0
idx = np.random.permutation(len(X))
X = X[idx]
y = y[idx]

In [19]:
net = Net(3, 32, 512)
par = sum(p.numel() for p in net.parameters() if p.requires_grad)

print("The network contains {} parameters.".format(par))

The network contains 2712066 parameters.


In [ ]:
batch_size = 64
learning_rate = 0.005

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
result = train(net, X, y, optimizer, criterion, batch_size, learning_rate)

epoch: 1
 [LOSS] TRAIN 0.6700874076824693 / VALID 0.6672149954926707
 [ACC] TRAIN 0.5871174234866842 / VALID 0.5951190238107226
epoch: 2
 [LOSS] TRAIN 0.6690453996799497 / VALID 0.6654292069856919
 [ACC] TRAIN 0.5909181836645484 / VALID 0.5985197039646348
epoch: 3
 [LOSS] TRAIN 0.6677214782532274 / VALID 0.6640255659741148
 [ACC] TRAIN 0.5938521037858796 / VALID 0.5991198239886396
epoch: 4
 [LOSS] TRAIN 0.6751243627142126 / VALID 0.6705143414132236
 [ACC] TRAIN 0.5832499833578204 / VALID 0.5887177435546714
epoch: 5
 [LOSS] TRAIN 0.6826956946507863 / VALID 0.6805772873896984
 [ACC] TRAIN 0.5785157031505642 / VALID 0.5789157831924012
epoch: 6
 [LOSS] TRAIN 0.6929893692731937 / VALID 0.6902996927982665
 [ACC] TRAIN 0.5737147429843596 / VALID 0.5747149430124443
epoch: 7
 [LOSS] TRAIN 0.6637925642915207 / VALID 0.6591035415110862
 [ACC] TRAIN 0.6040541441899868 / VALID 0.6079215843228251
epoch: 8
 [LOSS] TRAIN 0.6728129170612946 / VALID 0.6688992260312719
 [ACC] TRAIN 0.592451823725894 / VA

In [ ]:
plt.plot(range(result['final_epoch']), result["tloss"], label='training loss')
plt.plot(range(result['final_epoch']), result["vloss"], label='validation loss')
plt.axvline(x=result['best_epoch'], label='early stop', color='black')
plt.legend()
plt.show()

In [ ]:
plt.plot(range(result['final_epoch']), result["taccs"], label='training accuracy')
plt.plot(range(result['final_epoch']), result["vaccs"], label='validation accuracy')
plt.axvline(x=result['best_epoch'], label='early stop', color='black')
plt.legend()
plt.show()

In [ ]:
net = Net(3, 32, 512)
net.train(X, y, result['best_epoch'], optimizer, criterion, batch_size, learning_rate)
y = []
split = 10
step = int(len(submission) / split)
for i in range(0, len(submission), step):
    batch = torch.Tensor(submission[i : i + step])
    batch = batch.view(-1, 3, 64, 64)
    batch = batch.cuda()

    y += list(map(int, net.forward(batch).max(1)[1].cpu()))

y = pd.DataFrame(y)
y.index.name = "id"
y.columns = ["label"]
y = y.replace(0, "Cat")
y = y.replace(1, "Dog")
y.index += 1
y.to_csv("submission.csv")